In [ ]:
#imports and preprocessing, setting up train_loader
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Unsupervised-concept-centered-prototypical-parts/code

Mounted at /content/drive
/content/drive/MyDrive/Unsupervised-concept-centered-prototypical-parts/code


In [ ]:
import torch
from torchvision import transforms, datasets
import matplotlib.pyplot as plt

from models.classificationModel import ClassificationModel

In [ ]:
model = ClassificationModel(out_dim=1000, dataset='./dataset/data/CUB_200_2011')



/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 96.2MB/s]


In [ ]:
model.backbone.fc = torch.nn.Sequential(#wariant 1
    torch.nn.Linear(in_features=512, out_features=128, bias=True),
    torch.nn.Linear(in_features=128, out_features=20, bias=True),
)

In [ ]:

model.load_state_dict(torch.load('saved_models/test.pth'))

RuntimeError: ignored

In [ ]:
batch_size: int = 64

In [ ]:
#declare test_dataset and test_loader

In [ ]:
correct, numel = 0, 0
loss_fn = torch.nn.CrossEntropyLoss()
logs: dict = {
    "test_loss": [],
    "test_accuracy": []
}

for _ in range(10):
    with torch.no_grad():
        for x_test, y_test in test_loader:
            output = model(x_test)
            y_pred = torch.argmax(output, dim=1)
            correct += torch.sum(y_pred == y_test).item()
            numel += batch_size
            loss = loss_fn(output, y_test)
            logs['test_loss'].append(loss.item())
            logs['test_accuracy'].append(correct / numel)

In [ ]:


def show_results(orientation='horizontal', accuracy_bottom=None, loss_top=None, **histories):
    if orientation == 'horizontal':
        f, ax = plt.subplots(1, 2, figsize=(16, 5))
    else:
        f, ax = plt.subplots(2, 1, figsize=(16, 16))
    for i, (name, h) in enumerate(histories.items()):
        if len(histories) == 1:
            ax[0].set_title("Best test accuracy: {:.2f}%".format(
                max(h['test_accuracy']) * 100,
            ))
        else:
            ax[0].set_title("Accuracy")
        ax[0].plot(h['test_accuracy'], color='C%s' % i, label='%s test' % name)
        ax[0].set_xlabel('epochs')
        ax[0].set_ylabel('accuracy')
        if accuracy_bottom:
            ax[0].set_ylim(bottom=accuracy_bottom)
        ax[0].legend()

show_results(model=logs)